In [1]:
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url=url+filename, filename=filename)
    else:
        print ('%s file already present. Skipping download.' %filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

text8.zip file already present. Skipping download.
Found and verified text8.zip


In [3]:
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        words = tf.compat.as_str (f.read(f.namelist()[0])).split()
    return words

words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


In [5]:
vocabulary_size = 50000

def build_dataset(words):
    count = [['UNK', -1]] # count is a list of list
    collections.Counter()
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary) # Dictionary 1-1 karke construct ho rha hai..
                                           # Dictionary[first word in Count 'UNK'] = 1
                                           # Dictionary[second word in Count 'The'] = 2
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5242, 3084, 12, 6, 195, 2, 3135, 46, 59, 156]


In [6]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size%num_skips == 0
    assert num_skips <= 2*skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen = span)
    
    for _ in range(span):
        buffer.append (data[data_index])
        data_index = data_index + 1
        
    for i in range(batch_size // num_skips):
        target = skip_window # target label at the center of the buffer
        targets_to_avoid = [target]
        for j in range(num_skips):
            # While loop makes sure, we pick unique label each time
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            
            batch[i*num_skips + j] = buffer[skip_window]
            labels[i*num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels


print ('data', [reverse_dictionary[x] for x in data[:8]])

"""First things first SKIP here is JUMP, not skip...
    For the sentence, anarchism oriented as a term of abuse first,
    skip_window = aage piche kitne ka window dekhna hai.. 
    num_skips = a subset of skip_window, out of 2*skip_window words forward&backword, random num_skip words pick karo
    
    So if skip_window is 2, aage k 2 words and piche k 2 words are of interest
    and say num_skips is 2, then 4 possible aage-piche words me se, select DISTINCT 2 words
    and say num_skips is 4, then 4 possible aage-piche words me se, select DISTINCT 4 words
    
    num_skips cannot be greater than 2*skip_window"""
for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['anarchism', 'as', 'a', 'originated', 'as', 'term', 'a', 'of']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['term', 'originated', 'a', 'anarchism', 'of', 'originated', 'term', 'as']


In [9]:
# Now train a skip-gram model

batch_size = 128
"""I used to think embedding size will be size of 
entire vocabulary. But NO"""
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1
num_skips = 2

# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16
valid_window = 100 # Ab top 100 me se 16 choose karenge
valid_examples = np.array(random.sample(range(valid_window), valid_size))

# Label vector 128x1 ka hai.. usme se random 64 NEG WORDS k liye negative train karo
num_sampled = 64 #number of negative examples to sample

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):
    
    # INPUT DATA
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # VARIABLES
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    
    # What will be the shape of weights???
    softmax_weights = tf.Variable(
        tf.truncated_normal(shape=[vocabulary_size, embedding_size],
                               stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed,
                                   train_labels, num_sampled, vocabulary_size))
    
    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
      
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))


In [12]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += 1
        
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 1.000000
Nearest to that: nominative, youngster, malm, strengthen, jarry, transhumanism, midway, inherent,
Nearest to were: outposts, albret, while, biopsy, mayhew, soliah, adages, prairies,
Nearest to often: orcus, palembang, constructible, post, accumulated, acceptable, coeducational, shakur,
Nearest to with: clogged, sligo, lambasted, alamein, aerosol, arc, coincided, madame,
Nearest to would: aman, timbre, wisdom, bremerhaven, excreted, geddy, subway, titration,
Nearest to this: displayed, lutheranism, distortions, soliciting, lectionis, kid, counsellor, sevilla,
Nearest to one: rik, adenine, embarrassment, silo, uniformitarianism, husbands, dispel, oakenfold,
Nearest to their: fiftieth, jeans, brightest, elitist, sensations, perils, sifre, workgroups,
Nearest to so: fluorides, encyclopedic, downloadable, oscillators, ferrand, bala, fairest, canto,
Nearest to or: adulterated, rcito, socratic, troy, indication, staffordshire, detours, synthesised,

In [13]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

ValueError: array must not contain infs or NaNs